In [46]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('sample').getOrCreate()
sc = spark.sparkContext

In [96]:
data = [(1, 1, 'John', 5000),
        (2, 1, 'Alice', 6000),
        (3, 1, 'Bob', 7000),
        (4, 2, 'Charlie', 5500),
        (5, 2, 'David', 6500),
        (6, 3, 'Eva', 7500),
        (7, 3, 'Frank', 8000),
        (8, 3, 'Grace', 9000),
       (1, 1, 'John', 5000),
       (None, 1, 'Sam', 8000)]

columns = ['id', 'department', 'employee_name', 'salary']

df = spark.createDataFrame(data=data, schema=columns)
df.show()

+----+----------+-------------+------+
|  id|department|employee_name|salary|
+----+----------+-------------+------+
|   1|         1|         John|  5000|
|   2|         1|        Alice|  6000|
|   3|         1|          Bob|  7000|
|   4|         2|      Charlie|  5500|
|   5|         2|        David|  6500|
|   6|         3|          Eva|  7500|
|   7|         3|        Frank|  8000|
|   8|         3|        Grace|  9000|
|   1|         1|         John|  5000|
|NULL|         1|          Sam|  8000|
+----+----------+-------------+------+



In [98]:
df1 = df.filter(col('id').isNull()).drop()
df2 = df.subtract(df1)
df2.show()

+---+----------+-------------+------+
| id|department|employee_name|salary|
+---+----------+-------------+------+
|  1|         1|         John|  5000|
|  2|         1|        Alice|  6000|
|  3|         1|          Bob|  7000|
|  4|         2|      Charlie|  5500|
|  5|         2|        David|  6500|
|  6|         3|          Eva|  7500|
|  7|         3|        Frank|  8000|
|  8|         3|        Grace|  9000|
+---+----------+-------------+------+



In [111]:
display(df)

DataFrame[id: bigint, department: bigint, employee_name: string, salary: bigint]

In [102]:
df.subtract(df).show()

+---+----------+-------------+------+
| id|department|employee_name|salary|
+---+----------+-------------+------+
+---+----------+-------------+------+



In [97]:
df1.show()

+----+----------+-------------+------+
|  id|department|employee_name|salary|
+----+----------+-------------+------+
|NULL|         1|          Sam|  8000|
+----+----------+-------------+------+



In [25]:
#df = df.na.drop(subset='id').drop_duplicates()
df = df.na.drop(subset='id').dropDuplicates()
df.show()

+---+----------+-------------+------+
| id|department|employee_name|salary|
+---+----------+-------------+------+
|  1|         1|         John|  5000|
|  2|         1|        Alice|  6000|
|  3|         1|          Bob|  7000|
|  4|         2|      Charlie|  5500|
|  5|         2|        David|  6500|
|  6|         3|          Eva|  7500|
|  7|         3|        Frank|  8000|
|  8|         3|        Grace|  9000|
+---+----------+-------------+------+



In [16]:
name = input("Name:")
li = ['a', 'e', 'i', 'o', 'u']
for i in li:
	if i in name:
		print(i)

Name: Basudev


a
e
u


In [18]:
li1 = [i for i in li if i in name]
li1

['a', 'e', 'u']

In [26]:
window = Window.partitionBy('department').orderBy(col('salary').desc())
df1 = df.withColumn('rank_number', row_number().over(window))
df1 = df1.filter(col('rank_number') == 3)
df1.show()

+---+----------+-------------+------+-----------+
| id|department|employee_name|salary|rank_number|
+---+----------+-------------+------+-----------+
|  1|         1|         John|  5000|          3|
|  6|         3|          Eva|  7500|          3|
+---+----------+-------------+------+-----------+



In [28]:
new_row = Row(9, 2, 'Parkin', 8000)
columns = ('id', 'department', 'employee_name', 'rank_number')
df2 = spark.createDataFrame([new_row], schema = columns)
df2.show()

+---+----------+-------------+-----------+
| id|department|employee_name|rank_number|
+---+----------+-------------+-----------+
|  9|         2|       Parkin|       8000|
+---+----------+-------------+-----------+



In [29]:
df = df.union(df2)
df.show()

+---+----------+-------------+------+
| id|department|employee_name|salary|
+---+----------+-------------+------+
|  1|         1|         John|  5000|
|  2|         1|        Alice|  6000|
|  3|         1|          Bob|  7000|
|  4|         2|      Charlie|  5500|
|  5|         2|        David|  6500|
|  6|         3|          Eva|  7500|
|  7|         3|        Frank|  8000|
|  8|         3|        Grace|  9000|
|  9|         2|       Parkin|  8000|
+---+----------+-------------+------+



In [35]:
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)
broadcast_variable = sc.broadcast(rdd)

Traceback (most recent call last):
  File "/home/basu/spark/python/pyspark/broadcast.py", line 183, in dump
    pickle.dump(value, f, pickle_protocol)
  File "/home/basu/spark/python/pyspark/rdd.py", line 386, in __getnewargs__
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [RDD_TRANSFORM_ONLY_VALID_ON_DRIVER] It appears that you are attempting to broadcast an RDD or reference an RDD from an 
action or transformation. RDD transformations and actions can only be invoked by the 
driver, not inside of other transformations; for example, 
rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values 
transformation and count action cannot be performed inside of the rdd1.map 
transformation. For more information, see SPARK-5063.


PicklingError: Could not serialize broadcast: PySparkRuntimeError: [RDD_TRANSFORM_ONLY_VALID_ON_DRIVER] It appears that you are attempting to broadcast an RDD or reference an RDD from an 
action or transformation. RDD transformations and actions can only be invoked by the 
driver, not inside of other transformations; for example, 
rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values 
transformation and count action cannot be performed inside of the rdd1.map 
transformation. For more information, see SPARK-5063.

In [38]:
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)

# Broadcast the RDD
broadcast_variable = sc.broadcast(rdd.collect())

In [39]:
broadcast_variable.value

[1, 2, 3, 4, 5]

In [47]:
data = [
    Row(1, 'Basu'),
    Row(2, 'Prashant'),
    Row(3, 'Agnibesh')
]
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True)])
df = spark.createDataFrame(data=data, schema=schema)
df.show()

+---+--------+
| id|    name|
+---+--------+
|  1|    Basu|
|  2|Prashant|
|  3|Agnibesh|
+---+--------+



In [48]:
data = [
    Row(1, 'Dev'),
    Row(5, 'Bala'),
    Row(3, 'Agni')
]
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True)])
df1 = spark.createDataFrame(data=data, schema=schema)
df1.show()

+---+----+
| id|name|
+---+----+
|  1| Dev|
|  5|Bala|
|  3|Agni|
+---+----+



In [49]:
b_v = broadcast(df)

In [57]:
df1.join(b_v, 'id').show()

+---+----+--------+
| id|name|    name|
+---+----+--------+
|  1| Dev|    Basu|
|  3|Agni|Agnibesh|
+---+----+--------+



In [64]:
df1.join(b_v, 'id', 'leftsemi').show()

+---+----+
| id|name|
+---+----+
|  1| Dev|
|  3|Agni|
+---+----+



In [67]:
df1.join(b_v, 'id', 'left').show()

+---+----+--------+
| id|name|    name|
+---+----+--------+
|  1| Dev|    Basu|
|  5|Bala|    NULL|
|  3|Agni|Agnibesh|
+---+----+--------+



In [68]:
df1.join(b_v, 'id', 'right').show()

24/02/03 22:14:26 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for right outer join.


+---+----+--------+
| id|name|    name|
+---+----+--------+
|  1| Dev|    Basu|
|  2|NULL|Prashant|
|  3|Agni|Agnibesh|
+---+----+--------+



24/02/03 22:14:27 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for right outer join.


In [70]:
df1.join(b_v, 'id', 'right_outer').show()

24/02/03 22:16:34 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for right outer join.


+---+----+--------+
| id|name|    name|
+---+----+--------+
|  1| Dev|    Basu|
|  2|NULL|Prashant|
|  3|Agni|Agnibesh|
+---+----+--------+



24/02/03 22:16:34 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for right outer join.


In [71]:
df1.join(b_v, 'id', 'outer').show()

24/02/03 22:16:51 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for full outer join.
24/02/03 22:16:52 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for full outer join.
24/02/03 22:16:52 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build right for full outer join.


+---+----+--------+
| id|name|    name|
+---+----+--------+
|  1| Dev|    Basu|
|  2|NULL|Prashant|
|  3|Agni|Agnibesh|
|  5|Bala|    NULL|
+---+----+--------+



In [66]:
help(df.join)

Help on method join in module pyspark.sql.dataframe:

join(other: 'DataFrame', on: Union[str, List[str], pyspark.sql.column.Column, List[pyspark.sql.column.Column], NoneType] = None, how: Optional[str] = None) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Joins with another :class:`DataFrame`, using the given join expression.
    
    .. versionadded:: 1.3.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    other : :class:`DataFrame`
        Right side of the join
    on : str, list or :class:`Column`, optional
        a string for the join column name, a list of column names,
        a join expression (Column), or a list of Columns.
        If `on` is a string or a list of strings indicating the name of the join column(s),
        the column(s) must exist on both sides, and this performs an equi-join.
    how : str, optional
        default ``inner``. Must be one of: ``inner``, ``cross``, ``outer``,
      

In [83]:
df1.unionByName(df2).show()

AnalysisException: Cannot resolve column name "name" among (id, department, employee_name, rank_number).

In [84]:
# Create DataFrames with different column names
df1 = spark.createDataFrame([[5, 2, 6]], ["col0", "col1", "col2"])
df2 = spark.createDataFrame([[6, 7, 3]], ["col1", "col2", "col3"])

# Using allowMissingColumns
df3 = df1.unionByName(df2, allowMissingColumns=True)
df3.printSchema
df3.show()

+----+----+----+----+
|col0|col1|col2|col3|
+----+----+----+----+
|   5|   2|   6|NULL|
|NULL|   6|   7|   3|
+----+----+----+----+



In [80]:
df2.printSchema()

root
 |-- id: long (nullable = true)
 |-- department: long (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- rank_number: long (nullable = true)



In [107]:
li = range(1, 10)
rdd = sc.parallelize(li)
rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]